 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P TSX 60 Index.

In [15]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp_tsx_history.csv`: Contains historical closing prices of the S&P TSX 60 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data.

In [16]:
whale_returns = Path("./Resources/whale_returns.csv")

In [17]:
# Reading whale returns
df_whalereturns = pd.read_csv(whale_returns, index_col="Date")

df_whalereturns.head()

,SOROS FUND MANAGEMENT LLC,PAULSON & CO.INC.,TIGER GLOBAL MANAGEMENT LLC,BERKSHIRE HATHAWAY INC
Date,,,,
2015-03-02,NaN,NaN,NaN,NaN
2015-03-03,-0.001266,-0.004981,-0.000496,-0.006569
2015-03-04,0.002230,0.003241,-0.002534,0.004213
2015-03-05,0.004016,0.004076,0.002355,0.006726
2015-03-06,-0.007905,-0.003574,-0.008481,-0.013098


In [18]:
# Count nulls
df_whalereturns.isnull().sum()

SOROS FUND MANAGEMENT LLC      1
PAULSON & CO.INC.              1
TIGER GLOBAL MANAGEMENT LLC    1
BERKSHIRE HATHAWAY INC         1
dtype: int64

In [19]:
# Drop nulls
df_whalereturns = df_whalereturns.dropna()
df_whalereturns.isnull().sum()

SOROS FUND MANAGEMENT LLC      0
PAULSON & CO.INC.              0
TIGER GLOBAL MANAGEMENT LLC    0
BERKSHIRE HATHAWAY INC         0
dtype: int64

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data.

In [20]:
# Reading algorithmic returns
algoreturns = Path("./Resources/algo_returns.csv")
df_algoreturns = pd.read_csv(algoreturns, index_col="Date")
df_algoreturns.head()

,Algo 1,Algo 2
Date,,
2014-05-28,0.001745,NaN
2014-05-29,0.003978,NaN
2014-05-30,0.004464,NaN
2014-06-02,0.005692,NaN
2014-06-03,0.005292,NaN


In [21]:
# Count nulls
df_algoreturns.isnull().sum()

Algo 1    0
Algo 2    6
dtype: int64

In [22]:
# Drop nulls
df_algoreturns = df_algoreturns.dropna()
df_algoreturns.isnull().sum()

Algo 1    0
Algo 2    0
dtype: int64

## S&P TSX 60 Returns

Read the S&P TSX 60 historic closing prices and create a new daily returns DataFrame from the data. 

In [23]:
# Reading S&P TSX 60 Closing Prices
sp_tsx60_returns = Path("./Resources/sp_tsx_history.csv")

sp_tsx60_returns = pd.read_csv(sp_tsx60_returns, index_col = 'Date')

sp_tsx60_returns.head()

,Close
Date,
1-Oct-2012,"$12,370.19"
2-Oct-2012,"$12,391.23"
3-Oct-2012,"$12,359.47"
4-Oct-2012,"$12,447.68"
5-Oct-2012,"$12,418.99"


In [24]:
# Check Data Types

sp_tsx60_returns.index = pd.to_datetime(sp_tsx60_returns.index)

sp_tsx60_returns

,Close
Date,
2012-10-01,"$12,370.19"
2012-10-02,"$12,391.23"
2012-10-03,"$12,359.47"
2012-10-04,"$12,447.68"
2012-10-05,"$12,418.99"
...,...
2019-12-20,"$17,118.44"
2019-12-23,"$17,128.71"
2019-12-24,"$17,180.15"


In [25]:
# Fix Data Types
sp_tsx60_returns['Close']=sp_tsx60_returns['Close'].str.replace('$','')

sp_tsx60_returns['Close']=sp_tsx60_returns['Close'].str.replace(',','')

sp_tsx60_returns['Close']=sp_tsx60_returns['Close'].astype('float')

sp_tsx60_returns

,Close
Date,
2012-10-01,12370.19
2012-10-02,12391.23
2012-10-03,12359.47
2012-10-04,12447.68
2012-10-05,12418.99
...,...
2019-12-20,17118.44
2019-12-23,17128.71
2019-12-24,17180.15


In [26]:
# Calculate Daily Returns
dailyreturns = sp_tsx60_returns.pct_change()

dailyreturns

,Close
Date,
2012-10-01,NaN
2012-10-02,0.001701
2012-10-03,-0.002563
2012-10-04,0.007137
2012-10-05,-0.002305
...,...
2019-12-20,0.003188
2019-12-23,0.000600
2019-12-24,0.003003


In [27]:
# Drop nulls
dropnulls = sp_tsx60_returns.dropna()
dropnulls.isnull().sum()

Close    0
dtype: int64

In [28]:
# Rename `Close` Column to be specific to this portfolio.
sp_tsx60_returns.rename(columns={'close':'S&P TSX 60'}, inplace=True)
sp_tsx60_returns.head()

,Close
Date,
2012-10-01,12370.19
2012-10-02,12391.23
2012-10-03,12359.47
2012-10-04,12447.68
2012-10-05,12418.99


## Combine Whale, Algorithmic, and S&P TSX 60 Returns

In [29]:
# Join Whale Returns, Algorithmic Returns, and the S&P TSX 60 Returns into a single DataFrame with columns for each portfolio's returns.
daily_returns = pd.concat([df_whalereturns, df_algoreturns, sp_tsx60_returns], axis='columns', join='inner') 
daily_returns.head()

,SOROS FUND MANAGEMENT LLC,PAULSON & CO.INC.,TIGER GLOBAL MANAGEMENT LLC,BERKSHIRE HATHAWAY INC,Algo 1,Algo 2,Close
Date,,,,,,,


---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

#### Calculate and Plot cumulative returns.

In [ ]:
# Plot daily returns of all portfolios

daily_returns.plot(legend=True, figsize = (25,10), title='Daily Returns')

In [ ]:
# Calculate cumulative returns of all portfolios

cumulative_returns = (1+daily_returns).cumprod()

# Plot cumulative returns
cumulative_returns.plot(legend=True, figsize = (25,12), title='Cummulative Returns')

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios.
4. Determine which portfolios are riskier than the S&P TSX 60.
5. Calculate the Annualized Standard Deviation.

In [ ]:
# Box plot to visually show risk
daily_returns.plot(kind = 'box',figsize=(25,10), title='Daily Returns')


### Create a box plot for each portfolio


### Calculate Standard Deviations

In [55]:
# Calculate the daily standard deviations of all portfolios
daily_std = daily_returns.std()

daily_std

SOROS FUND MANAGEMENT LLC     NaN
PAULSON & CO.INC.             NaN
TIGER GLOBAL MANAGEMENT LLC   NaN
BERKSHIRE HATHAWAY INC        NaN
Algo 1                        NaN
Algo 2                        NaN
Close                         NaN
dtype: float64

### Determine which portfolios are riskier than the S&P TSX 60

In [ ]:
# Calculate  the daily standard deviation of S&P TSX 60
daily_returns['S&P TSX 60'].std()
# Determine which portfolios are riskier than the S&P TSX 60
riskier_portfolios = []
for portfolio in portfolios_list:
    if daily_returns[portfolio].std() > daily_returns['S&P TSX 60'].std():
        riskier_portfolios.append(portfolio)
        
riskier_portfolios

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annual_std = daily_returns * np.sqrt(252)

annual_std

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window.
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P TSX 60.
3. Choose one portfolio, then calculate and plot the 60-day rolling beta for it and the S&P TSX 60.

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
rolling_std = daily_returns.rolling(window=21).std()

rolling_std

# Plot the rolling standard deviation
rolling_std.plot(figsize = (21,10), title='21-Day rolling standard deviation')

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
portfolio_corr = daily_returns.corr()
# Display de correlation matrix
portfolio_corr

### Calculate and Plot Beta for a chosen portfolio and the S&P 60 TSX

In [ ]:
# Calculate covariance of a single portfolio
covariance = daily_returns['S&P 60 TSX'].rolling(60).cov
covariance
# Calculate variance of S&P TSX
variance = daily_returns['S&P 60 TSX'].rolling(60).var()
variance
# Computing beta
beta = covariance / variance
beta
# Plot beta trend
beta.plot(figsize=(25,10),title = 'S&P 60 TSX')

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window

portfolio_std_ewm = daily_returns.ewm(com=21).std()
portfolio_std_ewm

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
portfolio_sharpe = (daily_returns.mean() * 252)/((daily_returns.std())*np.sqrt(252))

portfolio_sharpe


In [ ]:
# Visualize the sharpe ratios as a bar plot
portfolio_sharpe.plot(kind='bar', figsize=(21,3), title='Sharpe Ratios')

### Determine whether the algorithmic strategies outperform both the market (S&P TSX 60) and the whales portfolios.

Write your answer here! volatile

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P TSX 60. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock.
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns.
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others.
5. Include correlation analysis to determine which stocks (if any) are correlated.

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
#used stocks for analysis are SHOP, BAC OETX from Google finance to generate stock price
stocks_list = ['SHOP','BAC','OTEX']

def daily_returns():
    dfs = []
    for stock in stocks_list:
         file_path = Path("../Resources/historical.csv")
        stock_df = pd.read_csv(file_path, index_col = 'Date') 
        stock_df.rename(columns = {'Close':(f'{stock}')}, inplace = True)
        stock_df.index = pd.to_datetime(stock_df.index).date 
        dfs.append(stock_df)
        
    stocks_daily_price = pd.concat(dfs, axis = 'columns', join = 'inner') 
    stocks_daily_price.drop(columns = 'Symbol', inplace = True) 
    stocks_daily_returns.dropna(inplace = True) 
    return stocks_daily_returns
my_stocks_daily_returns = daily_returns()
my_stocks_daily_returns

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights : Have allocated equal weights to the stocks 0.33 
def equal_weights():
    allocation = []
    for stock in stocks_list:
        weight = 1 / len(stocks_list)
        allocation.append(weight)
    return allocation    
weights = equal_weights()
weights
# Calculate portfolio return
daily_returns = pd.DataFrame(daily_returns.dot(weights))
# Display sample data
daily_returns.rename(columns = {0:'MY PORTFOLIO'}, inplace = True)
daily_returns

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [113]:
# Join your returns DataFrame to the original returns DataFrame
daily_returns = pd.concat([daily_returns, daily_returns], axis = 'columns', join ='outer')

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
new_combined_daily_returns.dropna(inplace=True)

new_combined_daily_returns

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`
new_combined_annual_std = (new_combined_daily_returns.std())*(np.sqrt(252))

new_combined_annual_std

### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation
new_combined_rolling_std = new_combined_daily_returns.rolling(window = 21).std()
# Plot rolling standard deviation
new_combined_rolling_std.plot(figsize=(21,10))

### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation
new_combined_portfolios_corr = new_combined_daily_returns.corr()

new_combined_portfolios_corr

### Calculate and Plot the 60-day Rolling Beta for Your Portfolio compared to the S&P 60 TSX

In [ ]:
# Calculate and plot Beta
portfolio_rolling_cov = new_combined_daily_returns['MY PORTFOLIO'].rolling(window=60).cov(new_combined_daily_returns['S&P TSX 60'])

rolling_var = new_combined_daily_returns['S&P TSX 60'].rolling(window=60).var()

portfolio_rolling_beta.plot(figsize=(21,10), title = 'My Portfolio 60-Day Rolling Beta compared to S&P TSX 60')

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios
new_combined_portfolios_sharpe = round((new_combined_daily_returns.mean() * 252) / (new_combined_annual_std),1)

new_combined_portfolios_sharpe

In [ ]:
# Visualize the sharpe ratios as a bar plot
new_combined_portfolios_sharpe.plot(kind = 'bar', figsize = (21,3), title='Sharpe Ratios')

### How does your portfolio do?

Write your answer here!

In [ ]:
Analysed based on 3 stocks.